# Splatoon2のシーン分析

## モチベーション

録画を切り出す際の境界検出ができたらハッピー

In [1]:
import os
import shutil
import random
import tensorflow as tf
import keras
from keras.layers import Conv2D, MaxPooling2D, Input, Dense

tf.test.gpu_device_name()

Using TensorFlow backend.


'/device:GPU:0'

In [2]:
# 環境変数とか

# 元データ保存先
dataset_base_path = '.\\splat-scene-dataset'
dataset_split_base_path = '.\\dataset'

# データセットの分離比率
learn_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2

In [15]:
# learn/validate/testに分離
def split_dataset(src_base_path, dst_base_path, ratios):
    dataset_learn_path = os.path.join(dst_base_path, 'learn')
    dataset_val_path   = os.path.join(dst_base_path, 'val')
    dataset_test_path  = os.path.join(dst_base_path, 'test')
    pathes = [dataset_learn_path, dataset_val_path, dataset_test_path]
    # すでに作成されている場合は一旦削除
    if (os.path.exists(dst_base_path)):
        shutil.rmtree(dst_base_path)
    # ディレクトリ作成
    os.mkdir(dst_base_path)
    for p in pathes:
        os.mkdir(p)
    # ディレクトリ一覧取得
    categories = list(filter(lambda x: 
                             os.path.isdir(os.path.join(src_base_path, x)) 
                             and not(x.startswith('.')),
                     os.listdir(dataset_base_path)))
    print(categories)
    # 順番にコピーしてく
    for c in categories:
        files = os.listdir(os.path.join(src_base_path, c))
        files_count = len(files)
        random.shuffle(files)
        
        ratio_sum = sum(ratios)
        take_count = [int(files_count * (r / ratio_sum)) for r in ratios]
        print(c, files_count, take_count)

        for p in pathes:
            dst = os.path.join(p, c)
            print('Mkdir {}'.format(dst))
            os.mkdir(dst)
            
        count = 0
        for t, p in zip(take_count, pathes):
            target_files = files[count:count + t]
            print(c, t, p, len(target_files))
            src = [os.path.join(src_base_path, c, tf) for tf in target_files]
            dst = [os.path.join(p, c, tf) for tf in target_files]
            for s, d in zip(src, dst):
                # print('Copy {} -> {}'.format(s, d))
                shutil.copyfile(s, d)
            count = t
        print('{} copy {} files'.format(c, count))
        
split_dataset(dataset_base_path, dataset_split_base_path, [learn_ratio, val_ratio, test_ratio])

['battle', 'battle_finish', 'battle_loby', 'battle_matching', 'battle_result', 'battle_rule', 'battle_start', 'loading', 'menu', 'other', 'salmon', 'salmon_lobby', 'salmon_matching', 'salmon_miss', 'salmon_result', 'salmon_start', 'weapon_select']
battle 9558 [5734, 1911, 1911]
Mkdir .\dataset\learn\battle
Mkdir .\dataset\val\battle
Mkdir .\dataset\test\battle
battle 5734 .\dataset\learn 5734
battle 1911 .\dataset\val 1911
battle 1911 .\dataset\test 1911
battle copy 1911 files
battle_finish 215 [129, 43, 43]
Mkdir .\dataset\learn\battle_finish
Mkdir .\dataset\val\battle_finish
Mkdir .\dataset\test\battle_finish
battle_finish 129 .\dataset\learn 129
battle_finish 43 .\dataset\val 43
battle_finish 43 .\dataset\test 43
battle_finish copy 43 files
battle_loby 280 [168, 56, 56]
Mkdir .\dataset\learn\battle_loby
Mkdir .\dataset\val\battle_loby
Mkdir .\dataset\test\battle_loby
battle_loby 168 .\dataset\learn 168
battle_loby 56 .\dataset\val 56
battle_loby 56 .\dataset\test 56
battle_loby copy